In [7]:
import pandas as pd
import numpy as np
import json

In [8]:
#Path here

base_path = 'data/created/'
sp500_path = 'data/sp/^GSPC.csv'
created_data_path = base_path + 'data500.h5'
technical500_path = base_path + 'technical500.h5'
news_path = '/data/news/'

In [9]:
df = pd.read_hdf(technical500_path, "data")
#df.index = pd.to_datetime(df.index, format='%Y-%m-%d')
#df.dropna(axis=0, inplace=True)
df.head()

,Open,High,Low,Close,Adj Close,Volume,lrets,MACD,stochastics,ATR
Date,,,,,,,,,,
1990-01-04,358.760010,358.760010,352.890015,355.670013,355.670013,177000000,-0.008650,0.000000,NaN,5.869995
1990-01-05,355.670013,355.670013,351.350006,352.200012,352.200012,158530000,-0.009804,-0.036878,NaN,4.320007
1990-01-08,352.200012,354.239990,350.540009,353.790009,353.790009,140110000,0.004504,-0.005628,NaN,3.699981
1990-01-09,353.829987,354.170013,349.609985,349.619995,349.619995,155210000,-0.011857,-0.100883,NaN,4.560028
1990-01-10,349.619995,349.619995,344.320007,347.309998,347.309998,175990000,-0.006629,-0.198862,NaN,5.299988


# Import News data and convert it from Json to pandas dataframe

In [10]:
def open_json(year, month):
    "Save as pandas dataframe"
    file_str = news_path + str(year) + '-' + '{:02}'.format(month) + '.json'
    with open(file_str) as data_file:    
        NYTimes_data = json.load(data_file)
    
    date_list = []
    df = pd.DataFrame()  
    df['News'] = None
    

    for i in range(len(NYTimes_data["response"]["docs"][:])):
        if NYTimes_data["response"]["docs"][i]["pub_date"][:10] not in df.index:
            df.loc[NYTimes_data["response"]["docs"][i]["pub_date"][:10]] = NYTimes_data["response"]["docs"][:][i]['headline']['main']
        else:
            df.loc[NYTimes_data["response"]["docs"][i]["pub_date"][:10]] = df.loc[NYTimes_data["response"]["docs"][i]["pub_date"][:10]].values + NYTimes_data["response"]["docs"][:][i]['headline']['main']
    
    df.index = pd.to_datetime(df.index, format='%Y-%m-%d')
    df.sort_index(inplace=True)
    
    return df

# Merge technical data with News

In [11]:
def merge_news_price(df):
    all_news = pd.DataFrame()
    for i in range(1990, 2019):
        for j in range(1, 13):
            try:
                temp = open_json(i, j)
                all_news = all_news.append(temp)
                print("Completed {}-{}".format(i, j))
            except:
                pass
            
    all_news = pd.concat([df, all_news], axis=1)
    
    return all_news

In [12]:
data = merge_news_price(df)

In [13]:
data.tail(30)

,Open,High,Low,Close,Adj Close,Volume,lrets,MACD,stochastics,ATR
Date,,,,,,,,,,
2018-08-29,2900.620117,2916.500000,2898.399902,2914.040039,2914.040039,2791860000,0.005685,5.325410,100.000000,18.979980
2018-08-30,2908.939941,2912.459961,2895.219971,2901.129883,2901.129883,2802180000,-0.004440,5.323926,86.505523,18.820068
2018-08-31,2898.370117,2906.320068,2891.729980,2901.520020,2901.520020,2880260000,0.000134,4.991565,86.913317,14.590088
2018-09-04,2896.959961,2900.179932,2885.129883,2896.719971,2896.719971,3077060000,-0.001656,4.058180,81.896015,16.390137
2018-09-05,2891.590088,2894.209961,2876.919922,2888.600098,2888.600098,3241250000,-0.002807,2.487810,65.317100,19.800049
2018-09-06,2888.639893,2892.050049,2867.290039,2878.050049,2878.050049,3139590000,-0.003659,0.341082,43.686587,24.760010
2018-09-07,2868.260010,2883.810059,2864.120117,2871.679932,2871.679932,2946270000,-0.002216,-1.843844,25.762245,19.689942
2018-09-10,2881.389893,2886.929932,2875.939941,2877.129883,2877.129883,2731400000,0.001896,-3.135264,35.313498,15.250000
2018-09-11,2871.570068,2892.520020,2866.780029,2887.889893,2887.889893,2899660000,0.003733,-3.370631,54.170840,25.739991


# Export new data as a csv file

In [14]:
data.to_hdf(created_data_path, 'data')